In [6]:

from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
from datetime import datetime
from win32com.client import Dispatch

ts=time.time()
date=datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
timestamp=datetime.fromtimestamp(ts).strftime("%H:%M:%S")

def speak(str1):
    speak=Dispatch(("SAPI.SpVoice"))
    speak.Speak(str1)

video=cv2.VideoCapture(0)
facedetect=cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

with open('data/names.pkl', 'rb') as w:
    LABELS=pickle.load(w)
with open('data/faces_data.pkl', 'rb') as f:
    FACES=pickle.load(f)
    FACES = FACES / 255.0
    
print('Shape of Faces matrix --> ', FACES.shape)

knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

imgBackground=cv2.imread("background.png")

COL_NAMES = ['NAME', 'TIME']

while True:
    ret,frame=video.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray, 1.3 ,5)
    for (x,y,w,h) in faces:
        crop_img=frame[y:y+h, x:x+w, :]
        resized_img = cv2.resize(crop_img, (50, 50)).flatten().reshape(1, -1) / 255.0
        distances, indices = knn.kneighbors(resized_img)
      # print("Distances:", distances, "Output:", output)
        threshold = 10                                                                      
        if distances[0][0] > threshold:
            output = ["Unknown"]
        else:
            output = knn.predict(resized_img)
            
        exist=os.path.isfile("Attendance/Attendance_" + date + ".csv")
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.rectangle(frame,(x,y-35),(x+w,y),(0,128,255),-1)
        cv2.putText(frame, str(output[0]), (x,y-15), cv2.FONT_HERSHEY_COMPLEX, 0.9, (255,255,255), 1)        
        attendance=[str(output[0]), str(timestamp)]
    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame",imgBackground)
    k=cv2.waitKey(1)
    if k == ord('o') and output[0] != "Unknown":
        speak("Attendance Taken..")
        time.sleep(3)
        if exist:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(attendance)
            csvfile.close()
        else:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
            csvfile.close()
    if k==ord('q'):
        break
video.release()
cv2.destroyAllWindows()



Shape of Faces matrix -->  (100, 7500)


In [2]:
import pickle
with open('data/names.pkl', 'rb') as w:
    LABELS = pickle.load(w)
    print(LABELS)

['Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush', 'Piyush']

In [3]:
with open('data/faces_data.pkl', 'rb') as f:
    FACES=pickle.load(f)
    print(FACES)

[[ 21  22  15 ...  45  54  53]
 [ 32  27  24 ...  35  42  42]
 [ 31  30  24 ...  26  39  39]
 ...
 [127 131 130 ...  33  45  47]
 [124 127 128 ...  31  39  39]
 [127 129 124 ...  38  46  49]]
